In [3]:
import numpy as np
from google import genai
from pypdf import PdfReader

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-3-flash-preview", 
    contents="Hur lägger man ett avkast i handboll?."
)

print(response.text)

Att lägga ett avkast i handboll kan verka enkelt, men det finns specifika regler för hur det ska gå till, särskilt efter de regeländringar som infördes 2022.

Här är en steg-för-steg-guide:

### 1. Placering
*   **Var:** Avkastet tas från mitten av planen (mittlinjen). 
*   **Den nya regeln (Avkastsfältet):** I modern handboll (på högre nivå) använder man ofta en **avkastzon**. Det är en cirkel i mitten av planen med en diameter på 4 meter. Spelaren som ska kasta måste befinna sig inom denna cirkel.
*   **Traditionellt:** Om det inte finns en markerad cirkel ska spelaren stå med en fot på mittlinjen.

### 2. Utförande
*   **Domarens signal:** Du får inte kasta bollen förrän domaren har blåst i pipan.
*   **Tidsgräns:** När domaren har blåst har du **3 sekunder** på dig att kasta bollen.
*   **Rörelse:** Du får röra dig inom avkastzonen/cirkeln (du behöver inte stå still), men du får inte hoppa över linjen eller springa ut ur zonen innan bollen har lämnat din hand.

### 3. Medspelarnas 

### Läsa in en pdf-fil

In [33]:
reader = PdfReader("spelregler_handboll_2025.pdf")

text = ""
for page in reader.pages:
    text += page.extract_text()

In [34]:
print(len(text)) # how many characters in the text

187232


In [35]:
print(text[17:550]) # print a part of the text


 
 
 
 
 
SPELREGLER 
Med kommentarer för 
HANDBOLL 
➢ Klarlägganden till spelreglerna 
 
➢ Reglemente rörande avbytarområdet 
 
➢ Reglemente gällande Shoot-Out 
 
➢ PM för domare, tidtagare, sekreterare 
och föreningar 
 
 
 
Utgivna av 
SVENSKA HANDBOLLFÖRBUNDET 
 
Gällande från den 1 juli 2025 
 
 
Eftertryck utan tillstånd är förbjudet 
 
4  
 
 
 
 
INNEHÅLLSFÖRTECKNING 
Regel 1: Spelplanen .............................................................................................. 5 
Regel 2: Speltiden, slutsignal och 


### Fixed length-chunking

In [36]:
# Split the text into chunks with overlap to preserve context

chunks = []
n = 1000
overlap = 200
for i in range(0, len(text), n - overlap):
    chunks.append(text[i:i + n])

print(f"Antal chunks: {len(chunks)}.")

Antal chunks: 235.


In [37]:
print(chunks[3]) # print the fourth chunk

Domartecken.......................................................................................................... 52 
Klarlägganden till spelreglerna ............................................................................ 55 
Reglemente rörande avbytarområdet ................................................................. 72 
Riktlinjer och tolkningar ...................................................................................... 75 
Riktlinjer för spelplan och mål............................................................................ 90 
Shoot-out - Regler och utförande....................................................................... 94 
Handbollsdomarens 10 punkter ......................................................................... 98 
PM för domare .................................................................................................... 100 
PM för tidtagare och sekreterare ................................................................

In [38]:
print(text[26:584]) # print a part of the text

 
SPELREGLER 
Med kommentarer för 
HANDBOLL 
➢ Klarlägganden till spelreglerna 
 
➢ Reglemente rörande avbytarområdet 
 
➢ Reglemente gällande Shoot-Out 
 
➢ PM för domare, tidtagare, sekreterare 
och föreningar 
 
 
 
Utgivna av 
SVENSKA HANDBOLLFÖRBUNDET 
 
Gällande från den 1 juli 2025 
 
 
Eftertryck utan tillstånd är förbjudet 
 
4  
 
 
 
 
INNEHÅLLSFÖRTECKNING 
Regel 1: Spelplanen .............................................................................................. 5 
Regel 2: Speltiden, slutsignal och timeout ..........................


### Embeddings

In [39]:
# Function to create embeddings for a given text chunk

from google.genai import types

def create_embeddings(text, model="text-embedding-004", task_type="SEMANTIC_SIMILARITY"): 
    return client.models.embed_content(model=model, contents=text, config=types.EmbedContentConfig(task_type=task_type))

In [40]:
# Create embeddings for each chunk (handle batches of max 100)
all_embeddings = []
batch_size = 100

for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i + batch_size]
    embeddings_response = create_embeddings(batch)
    all_embeddings.extend(embeddings_response.embeddings)

print(f"Skapade embeddings för {len(all_embeddings)} chunks")

Skapade embeddings för 235 chunks


In [41]:
len(all_embeddings)

all_embeddings[0].values[0:10]

[-0.06850545,
 0.06893879,
 -0.034853388,
 -0.008456322,
 0.058385666,
 0.014736956,
 0.03768798,
 0.035730246,
 0.022586556,
 -0.023995418]

### Semantisk sökning

In [42]:
def cosine_similarity(vec1, vec2):
    return (np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2)))

In [43]:
def semantic_search(query, chunks, embeddings, k=5):
    query_embedding = create_embeddings(query).embeddings[0].values 
    similarity_scores = []
    
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(query_embedding, chunk_embedding.values)
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]
    
    return [chunks[index] for index in top_indices]

In [44]:
fråga = "Hur stor ska en handboll vara?"
svar = semantic_search(fråga, chunks=chunks, embeddings=all_embeddings, k=1)
print(svar)

['llstorlekar avseende omkrets och vikt ska användas för de olika \nålderskategorierna: \n \n- 55,5-57,5 cm och 400–425 g (IHF storlek 3) för manliga spelare och för manlig \nungdom (16 år och äldre); \n- 51,5-53,5 cm och 300–325 g (IHF storlek 2) för kvinnliga spelare, för kvinnlig \nungdom (14 år och äldre) och manlig ungdom (12 till 16 år); \n- 49-51 cm och 290–315 g (IHF storlek 1) för kvinnlig ungdom (8 till 14 år) och \nmanlig ungdom (8 till 12 år). \n \nSHF:s kommentar: \nFör ungdomshandboll gäller tävlingsbestämmelser för barn och ungdom \n(TBBU). Beträffande klisterfria bollar (§ 3:2 b), se SHF:s tävlingsbestämmelser. \n \nIHF:s kommentar: \nDe tekniska kraven på bollarna som används i alla officiella matcher \nfinns i IHF:s bollreglemente. Storleken och vikten för bollar som \nanvänds för minihandboll är inte fastställda i de vanliga spelreglerna. \n3:3 Vid varje match ska minst två reglementsenliga bollar finnas tillgängliga. \nReservbollen måste omedelbart vara tillgänglig 

### Generera bra svar med RAG

In [45]:
system_prompt = """Jag kommer ställa dig en fråga, och jag vill att du svarar
baserat bara på kontexten jag skickar med, och ingen annan information.
Om det inte finns nog med information i kontexten för att svara på frågan,
säg "Det vet jag inte". Försök inte att gissa.
Formulera dig enkelt och dela upp svaret i fina stycken. """

In [46]:
def generate_user_prompt(query):
    context = "\n".join(semantic_search(query, chunks, all_embeddings))
    user_prompt = f"Frågan är {query}. Här är kontexten: {context}."
    return user_prompt

In [47]:
def generate_response(system_prompt, user_message, model="gemini-3-flash-preview"):
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=system_prompt),
            contents=generate_user_prompt(user_message)
        )
    return response

In [48]:
print(generate_response(system_prompt, "Hur stor ska en handboll vara?").text)

None


In [49]:
fråga = "Hur stor ska en basketboll vara?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Det vet jag inte.


In [50]:
fråga = "Hur gammal är man när man spelar ungdoms hanbdboll?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Baserat på texten spelar man ungdomshandboll i olika ålderskategorier från 8 år och uppåt.

När det gäller speltid för ungdomslag delas åldrarna upp i grupperna 8–12 år och 12–16 år.

För bollstorlekar delas ungdomar in i följande åldrar:
*   **Manlig ungdom:** 8–12 år, 12–16 år samt 16 år och äldre.
*   **Kvinnlig ungdom:** 8–14 år samt 14 år och äldre.


### Spara modellen för Streamlit

In [51]:
import pickle
import json

# Spara chunks som JSON
chunks_file = "handboll_chunks.json"
with open(chunks_file, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)
print(f"Chunks sparade i {chunks_file}")

# Spara embeddings som pickle
embeddings_file = "handboll_embeddings.pkl"
with open(embeddings_file, "wb") as f:
    pickle.dump(all_embeddings, f)
print(f"Embeddings sparade i {embeddings_file}")

print("Filerna är nu sparade och klara för Streamlit-appen!")

Chunks sparade i handboll_chunks.json
Embeddings sparade i handboll_embeddings.pkl
Filerna är nu sparade och klara för Streamlit-appen!
